In [11]:
from funcs.notebook import *

- false positive numbers are often double counted, not a reliable source of algorithm efficiency
- the vetted list is rather a search protocol
- 20s and 2min duplicate lcs >>> use only 20s except for four cases with marginal but real looking flares that are not detected in 20s but in 2min
- 


In [138]:
df = pd.read_csv("../results/2020_02_flares_vetted.csv")

In [141]:
df.note.unique()#.shape

array([nan, 'duplicate lc', 'not seen in TPF', 'end of LC sys?',
       'end of LC noise', 'checked TPF', 'lc sys', 'marginal detection',
       'unusual pattern, also second such event around 2070 not recovered',
       'supershort but real in TPF', 'De-trending failed',
       'end of LC sys', 'signal center is off pixelmask center',
       'only in LC, not in TPF?', 'right after transit',
       'saturated in detector', 'simultaneous FP in multiple Lcs',
       'start of lc', 'added the tail to this solution already',
       'tail of flare right before', 'entire TPF flashes', 'lc noise',
       'some sort of variability that is not flares', 'SSO?',
       'has tail, added manually at the end',
       'gradual like a spot, extended flare range manually', 'SSO',
       'De-trending failed, EB, no flares', 'a mess in the TPF',
       'gradual like a spot, saturated TPF?', 'cut off',
       'tail of possible flare right before', 'gradual like a spot',
       'cosmic ray flag', 'looks go

In [14]:
df[df.note == "saturated in detector"].shape[0]

547

In [15]:
df[df.note == "simultaneous FP in multiple Lcs"].shape[0]

1367

In [16]:
lcs_without_flare_candidates = df[df.ampl_rec.isnull()].shape[0]
flare_candidates = df[~df.ampl_rec.isnull()].shape[0]
assert df.shape[0] == flare_candidates + lcs_without_flare_candidates

In [17]:
flare_candidates, lcs_without_flare_candidates

(5652, 12487)

In [18]:
lcs_with_flare_candidates = df[~df.ampl_rec.isnull()].groupby(["ID","qcs","lc_n"]).first().shape[0]
lcs_with_flare_candidates

2413

In [48]:
a = df.loc[(df.qcs > 26) & (df.lc_n == 1) & (df["note"]!="duplicate lc"),["ID","qcs"]].drop_duplicates(keep="last")
a["has2min"] = True
a

,ID,qcs,has2min
12,GJ 1252,27,True
17,GJ 143,28,True
19,GJ 143,29,True
21,GJ 143,30,True
49,HATS-13,28,True
...,...,...,...
17698,TIC 306472057,28,True
17707,TIC 350332997,27,True
17709,TIC 350332997,28,True
17725,TIC 391904697,27,True


In [49]:
b = df.loc[(df.qcs > 26) & (df.lc_n == 0),["ID","qcs"]].drop_duplicates(keep="last")
b['has20s'] =True
b

,ID,qcs,has20s
11,GJ 1252,27,True
13,GJ 143,28,True
18,GJ 143,29,True
20,GJ 143,30,True
48,HATS-13,28,True
...,...,...,...
18132,TIC 299798795,28,True
18135,TIC 101230735,27,True
18136,TIC 278198753,27,True
18137,TIC 278683844,31,True


In [89]:
c = b.merge(a, how="inner").shape
c

(511, 4)

In [85]:
df[df.note == "duplicate lc"].shape

(136, 19)

In [130]:
r=df[(df.qcs > 26)].groupby(["ID","qcs","lc_n"]).first().reset_index()[["ID","qcs","lc_n"]].groupby(["ID","qcs"]).count()
r = r[r.lc_n==2].reset_index()

In [131]:
for j,v in r.iterrows():
    df.loc[(df.ID == v.ID) & (df.qcs == v.qcs) & (df.lc_n==1) ,"note"]= "duplicate lc"

In [134]:
df[df.note!="duplicate lc"], df.shape

(        istart    istop     cstart      cstop       tstart        tstop  \
 0          NaN      NaN        NaN        NaN          NaN          NaN   
 1          NaN      NaN        NaN        NaN          NaN          NaN   
 2          NaN      NaN        NaN        NaN          NaN          NaN   
 3          NaN      NaN        NaN        NaN          NaN          NaN   
 4          NaN      NaN        NaN        NaN          NaN          NaN   
 ...        ...      ...        ...        ...          ...          ...   
 18134   9986.0  10011.0  1309142.0  1309167.0  1008.467093  1008.484120   
 18135  68803.0  68890.0  3583826.0  3583913.0  2057.048922  2057.069061   
 18136   1590.0   1633.0  3495709.0  3495752.0  2036.647599  2036.657552   
 18137  77126.0  77152.0  4050732.0  4050758.0  2165.124274  2165.130293   
 18138  55177.0  55192.0  3790024.0  3790039.0  2104.780506  2104.783978   
 
           ed_rec  ed_rec_err  ampl_rec       dur  total_n_valid_data_points  \
 0    